In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import seaborn as sns
from ExplainableMaximumLikelihoodCalculator import ExplainableMaximumLikelihoodCalculator
import pysam
from Bio import SeqIO, Seq, SeqRecord, pairwise2
from Bio.pairwise2 import format_alignment
from LoadDataUtils import getListOfReadsFromBamFile, getListOfReadsFromFastaFile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
import shap
from scipy.stats import chisquare
from scipy.special import rel_entr
fileNameSapiens =  "data/simulatedData/human_chinese_AF346973_500samples.fas" #File that contains 500 reads from homo sapiens
fileNameNeanderthals =  "data/simulatedData/Neanderthal_Goyet_KX198085_500samples.fas" #File that contains 500 reads from neanderthal
fileNameDenisovans =  "data/simulatedData/denisova_kx663333_500samples.fas" #File that contains 500 reads from denisovan
neanderthals_500_generated = getListOfReadsFromFastaFile(fileNameNeanderthals)
sapiens_500_generated = getListOfReadsFromFastaFile(fileNameSapiens)
denisovan_500_samples = getListOfReadsFromFastaFile(fileNameDenisovans)
path_to_frequencies_table = "data/substitution_matrix.tsv"
import itertools
 
def findsubsets(s, n):
    subsets = list(itertools.combinations(s, n))
    return [list(a) for a in subsets]

2023-06-25 17:30:39.419501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 17:30:39.803917: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-25 17:30:39.826034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-25 17:30:39.826056: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
sapiens_reference_file_names = [
                    "data/reference_files/human_AF346981_French.fa",
                     "data/reference_files/human_AY195760_Korea.fa",
                      "data/reference_files/human_AY882416_Ethiopia.fa",
                      "data/reference_files/human_AY963586_Italian.fa",
                      "data/reference_files/human_AY195781_Caucasian.fa",
                      "data/reference_files/human_AY195757_Iraqi-Israeli.fa",
                      "data/reference_files/human_AY195749_NativeAmerican.fa"]
neanderthals_reference_file_names = [
                            "data/reference_files/neanderthal_mezmaiskaya1_FM865411.fa",
                           "data/reference_files/Neanderthal_Altai_KC879692.fa",
                           "data/reference_files/Neanderthal_Denisova11_full_mtDNA_KU131206.fa",
                           "data/reference_files/Neanderthal_Spy_94a_MG025538.fa",
                            "data/reference_files/Neanderthal_Vindija33.16_AM948965.fa",
                            "data/reference_files/Neanderthal_Vindija33.19_KJ533545.fa"]
denisovan_reference_file_names = [  
                        "data/reference_files/Denisova_MT576653.1.fa",
                        "data/reference_files/Denisova_MT576652.1.fa",
                        "data/reference_files/Denisova_4_FR695060.fa",
                        "data/reference_files/Denisova_8_KT780370.fa",
                        "data/reference_files/Denisova_manual_phalanx_NC_013993.fa",
                        "data/reference_files/Denisova_MT576651.1.fa"]

In [3]:
def generate_sample(sap, nean, den):
    list_before =  list(np.random.choice(sapiens_500_generated, sap)) + list(np.random.choice(neanderthals_500_generated, nean)) + list(np.random.choice(denisovan_500_samples, den))
    l = []
    for i in list_before:
        l.append(str(i))
    return l

In [37]:

size_of_sample = 50
number_of_sap = random.randint(0,size_of_sample)
number_of_nean = ((size_of_sample-number_of_sap)*2)//3
number_of_denisovan = size_of_sample - number_of_sap - number_of_nean
print(number_of_sap, number_of_nean, number_of_denisovan)
sample = generate_sample(number_of_sap, number_of_nean, number_of_denisovan)
print(sample)
maximum_likelihood_calculator_on_sample = ExplainableMaximumLikelihoodCalculator(sample,
                                            ref_neanderthal_file_names=neanderthals_reference_file_names,
                                            ref_sapien_file_names=sapiens_reference_file_names,
                                            ref_denisovan_file_names=denisovan_reference_file_names,
                                            path_to_substitution_matrix=path_to_frequencies_table,
                                            number_of_jobs=-1)

39 7 4
['TGTATTTCGTACATTACTGCCAGCCACCATGAATATTGCACGGTACCATAAATAC', 'ACGGACTTCACGTCATTATTGGCTTAACTTTCCTCACTATCTGCTTCATCCGCCAACTAATATTTCATTTT', 'TTATGAAACTTAAGGGTCGAAGGTGGATTTAGCAGTAAACTGAGAGTAGAGTGC', 'GTGGCTCCTTTAACCTCTCCACCCTTATCACAACACAAGAATACCTCTGATTACTCCTGCCATCATGACCCTTGGCCATAATAT', 'CGTTCAATATTACAGGCGAACATACTTACTAAAGTGTGTTAATTAATTAATGCTTGTAGGACATAATAATAACAATTG', 'CTACCCCGATGCATACACCACATGAAACATCCTATCATCTGTAGGCTCATTCATTTCTCTAACAGCAATAA', 'GTGACTATATGGATGCCCCCCACCCTACCACACATTTGAAGAACCCGTATACATAAAATCTAGACAAAAAAGGAAGGAATCGAACCCGCCAAAGT', 'TCACAAAAAACAATAGCCTCATCATCCCCACCATCATAGCCACCATCACCCTCCTTAACCTCTACTTCTACCTACGCTTAATCTACTCCACCT', 'TCAATACAACCCCCGCTCATCCTACCTAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCAAACCCCAAAAACACCC', 'TAATCTCTACTTCTACCTACGCCTAATCTACTCCACCTCAATCACACTACTCC', 'AGCATCACCAGTATTAGAGGCACCGCCTGCTCAGTGAT', 'CATGCCTCAGGATACTCCTCAATAGCCATCGCTGTAG', 'AAATCTCCTTAATTATAACATTCACAGCCACAGAATTAATCATATTTTATAT', 'TCCACGCCTTCTTCAAAGCCATACTATTTATGTGCTCCGGGTCCATCATCCACAACCTTAACAATGAACAAG

In [39]:
estimation = maximum_likelihood_calculator_on_sample.estimate_species_proportions(result_resolution=50)

In [40]:
estimation.values

array([[0.8 , 0.16, 0.04]])

In [41]:
est_b = maximum_likelihood_calculator_on_sample.estimate_species_proportions_gradient_descent(1)


In [42]:
est_b

,Homo Sapiens,Neanderthals,Denisovans
Estimation,0.8,0.16,0.04


In [24]:
(est_b.values - estimation.values).sum()

0.0

In [43]:
for i in range(100):
    val = maximum_likelihood_calculator_on_sample.estimate_species_proportions_gradient_descent(3).values
    diff = (est_b.values - val).sum()
    print(val)
    
    

[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 0.04]]
[[0.8  0.16 